# Config & Imports

In [1]:
config = {
    'data': {
        'n_datasets': 10_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 20, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        'n_informative': 10,
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        
        'n_targets': 2,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 2,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'noise': 0.01,
        # flip_y (fraction of samples whose class is assigned randomly)
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': None,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },    
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.2,
                'val_size': 0.1,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 150,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.2,
                'val_size': 0.1,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 100,
        'use_gpu': False,
        'gpu_numbers': '4',
        'RANDOM_SEED': 1,   
    }
}

In [2]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow import keras
import keras.layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU

from keras.regularizers import L1L2
from keras.regularizers import L2

import utilities_LR

2022-05-31 13:53:41.144317: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-31 13:53:41.144367: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = config['computation']['gpu_numbers'] if config['computation']['use_gpu'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if config['computation']['use_gpu'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if config['computation']['use_gpu'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if config['computation']['use_gpu'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  0
Num XLA-GPUs Available:  0


2022-05-31 13:54:08.248198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-31 13:54:08.248238: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-31 13:54:08.248273: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dws-02): /proc/driver/nvidia/version does not exist


# Functions

In [5]:
#def load_model(datasetIndex):
 #   # with open(utilities_LR.lambda_path(config, datasetIndex), 'r') as f:
  #  #     model = keras.models.load_model(f)
   # model = keras.models.load_model(utilities_LR.lambda_path(config, datasetIndex))
    #return model

In [6]:
# def get_weightdata(datasetIndex):
  #   weights_list = load_model(datasetIndex).get_weights()
    # return np.concatenate([x.flatten() for x in weights_list])

In [7]:
def load_lambda():
    directory = utilities_LR.lambda_path_LR(config)
    
    with open(directory + '/lambda_weights_list.npy', "rb") as f:
        return np.load(f, allow_pickle=True)

In [8]:
#def get_models():
#    # return np.array([get_weights(i) for i in range(config['data']['n_datasets'])])
#    # return np.array([get_weightdata(i) for i in range(10)])
#    with open(utilities_LR.lambda_path_LR(config, 0) + '.npy', 'rb') as f:
#        weights_list = np.load(f, allow_pickle=True)
#    for i in range(1, config['data']['n_datasets']):
#        with open(utilities_LR.lambda_path(config, i) + '.npy', 'rb') as f:
#            weights = np.load(f, allow_pickle=True)
#        weights_list = np.vstack([weights_list, weights])
#    return weights_list

In [9]:
def get_coef():
    return np.load(utilities_LR.data_path_LR(config) + '/coef_list_targetForInet.npy')

In [10]:
def train_test_val_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size=config['inets']['data_prep']['train_test_val_split']['test_size'] + config['inets']['data_prep']['train_test_val_split']['val_size'], 
                                                        random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
                                                        shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
                                                        stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])
    X_test, X_val, y__test, y_val = train_test_split(X_test, 
                                                    y_test, 
                                                    test_size=config['inets']['data_prep']['train_test_val_split']['val_size'] / (config['inets']['data_prep']['train_test_val_split']['test_size'] + config['inets']['data_prep']['train_test_val_split']['val_size']), 
                                                    random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])
    return X_train, X_test, X_val, y_train, y_test, y_val
    

In [11]:
def save_model(model):
    path = utilities_LR.inet_path_LR(config)
    
    Path(path).mkdir(parents=True, exist_ok=True)
    
    # weights_list = model.get_weights()
    
    # weights_linearized = np.concatenate([x.flatten() for x in weights_list])
    
    # with open(path + '/inet_weights.npy', "wb") as f:
        # np.save(f, weights_linearized, allow_pickle=True)
    model.save(path + '/modelKeras')

# Model Training

In [12]:
X = load_lambda()

In [13]:
X.shape

(10000, 8362)

In [14]:
y = get_coef()

In [15]:
y[0]

array([ 0.00495198, -0.01854766, -0.03919359,  0.04854344,  0.0091744 ,
       -0.07450365, -0.04318035,  0.03160309, -0.11410224, -0.01132964,
        0.00093653,  0.15812977, -0.24470446, -0.2362686 ,  0.00529975,
       -0.03122651, -0.50247647, -0.02148568,  0.04833143, -0.04019827])

In [16]:
y.shape

(10000, 20)

In [ ]:
# Data Prep
X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
                                                                      y)

# Model Def
model = Sequential()
# model.add(BatchNormalization(axis=1,
#    momentum=0.99,
#    epsilon=0.001,
#    center=True,
#    scale=True,
#    beta_initializer='zeros',
#    gamma_initializer='ones',
#    moving_mean_initializer='zeros',
#    moving_variance_initializer='ones',
#    beta_regularizer=None,
#    gamma_regularizer=None,
#    beta_constraint=None))
model.add(Dense(
    units=1000,
    kernel_regularizer=L1L2(l1=1e-5, l2=1e-4),
    bias_regularizer=L2(1e-4),
    activity_regularizer=L2(1e-5)
))
model.add(ReLU())
#model.add(Dense(6500, activation='linear'))
#model.add(Dense(6000, activation='relu'))
#model.add(Dense(5500, activation='linear'))
#model.add(Dense(3500, activation='relu'))
#model.add(Dense(2500, activation='relu'))
#model.add(Dense(1500, activation='linear'))
#model.add(Dense(500, activation='relu'))
model.add(Dense(y.shape[1], activation='relu'))


#opt = tf.keras.optimizers.Adam(
#    learning_rate=3e-4,
#    beta_1=0.999999,
#    beta_2=0.999,
#    epsilon=1e-07,
#    amsgrad=False,
#    name="Adam")

opt = tf.keras.optimizers.SGD(
    learning_rate=0.01,
    momentum=0.0,
    nesterov=False,
    name='SGD'
)

model.compile(optimizer=opt, loss='mse', metrics=['mse', keras.metrics.RootMeanSquaredError()])
model.build(input_shape=X_train.shape)
model.summary()

# Model fit
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=config['inets']['model_fit']['batch_size'],
                    epochs=config['inets']['model_fit']['epochs'],
                    verbose=config['inets']['model_fit']['verbose'],
                    callbacks=config['inets']['model_fit']['callbacks'],
                    validation_data=(X_val, y_val),
                    shuffle=config['inets']['model_fit']['shuffle'],
                    class_weight=config['inets']['model_fit']['class_weight'],
                    sample_weight=config['inets']['model_fit']['sample_weight'],
                    initial_epoch=config['inets']['model_fit']['initial_epoch'],
                    steps_per_epoch=config['inets']['model_fit']['steps_per_epoch'],
                    validation_steps=config['inets']['model_fit']['validation_steps'],
                    validation_batch_size=config['inets']['model_fit']['validation_batch_size'],
                    validation_freq=config['inets']['model_fit']['validation_freq'],
                   )
print(history.history['val_loss'])


2022-05-31 13:54:12.405390: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (6999, 1000)              8363000   
                                                                 
 re_lu (ReLU)                (6999, 1000)              0         
                                                                 
 dense_1 (Dense)             (6999, 20)                20020     
                                                                 
Total params: 8,383,020
Trainable params: 8,383,020
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
219/219 [==============================] - 14s 59ms/step - loss: 1.3197 - mse: 0.0821 - root_mean_squared_error: 0.2866 - val_loss: 1.3128 - val_mse: 0.0767 - val_root_mean_squared_error: 0.2770
Epoch 2/1000
219/219 [==============================] - 12s 54ms/step - loss: 1.3120 - mse: 0.0774 - r

In [ ]:
save_model(model)